<h1 align='center'> TITLE: Banknote Authentication Model </h1>

<h2 align='center'> UNIVERSITY OF ENGINEERING & MANAGEMENT KOLKATA </h2>
<h3 align='right'> GROUP ID: 28 </h3>
<h3 align='right'> CSE_3A_55 - Somdeep Jana - 12017009001009 </h3>
<h3 align='right'> CSE_3A_56 - Soumodip Roy - 12017009001422 </h3>


# Introduction
    This project is created for checking multiple classifiers on the UCI Banknote Authentication Data Set( https://archive.ics.uci.edu/ml/datasets/banknote+authentication).
    
    Everyday millions of people use banknotes to make transactions. The security of these banknotes for governments and banks is hence an essential factor to fight froud.

    Nowadays, sometimes it is too hard to spot counterfeit and genuine notes. Then, the aim of this example is to create a support system ready to help organizations to accurately classify fraudulent notes.

    Data were extracted from images that were taken from genuine and forged banknote-like specimens. For digitization, an industrial camera usually used for print inspection was used. The final images have 400x 400 pixels.

    Due to the object lens and distance to the investigated object gray-scale pictures with a resolution of about 660 dpi were gained. Wavelet Transform tool were used to extract features from images

In [1]:
from IPython.display import IFrame
IFrame('https://archive.ics.uci.edu/ml/datasets/banknote+authentication', width=800, height=450)

# Objective
    Main objective of this project is to evaluate, which classifier provides the best optimal solution for classifying whether a note is authentic or not.

# Importing Necessary Packages
    We are going to use python for this project. so first step is to getting all the necessary pckages

In [2]:
import numpy as np
np.__version__

'1.16.5'

In [3]:
import pandas as pd # for loading the CSV data
pd.__version__

'0.25.1'

In [42]:
from sklearn.model_selection import train_test_split # for spliting data into Training set and Test set
from sklearn.model_selection import GridSearchCV # for choosing the hyper paraenter
from sklearn.metrics import classification_report
from sklearn import metrics # for evaluating the models

In [5]:
from sklearn.neighbors import KNeighborsClassifier # for applying Knn classifier
from sklearn import svm # for applying svm
from sklearn.neural_network import MLPClassifier # for appling Neural Network

In [6]:
%matplotlib notebook
import matplotlib.pyplot as plt # for ploting the evaluating metrics

# Loading & Pre-processing The DataSet
    The data set is in CSV(Comma seperated Value) format. We are using pandas for importing this dataset

In [7]:
features_name= ["Variance", "Skewness", "Curtosis", "Entropy", "Class"]  # These are the features list for our dataset
banknote_data_set= pd.read_csv('data_banknote_authentication.csv', names= features_name)
banknote_data_set.head(5) # geting a snapshot of our data

,Variance,Skewness,Curtosis,Entropy,Class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


## Non-Numeric Values
    Our dataset dosen't contain any Non-Numeric value so we don't have to worry about that.


## Null Value
    Our dataset dose't have any null value so we don't have to wory about that.

## Standardization & Normalization
    Our dataset contains continious values so the values in all the columns are not in same range so we are performing Normalization and the scale across all the data are also not same so one value may get uncondition prority than others thats why we are performing Standardization.

In [8]:
print("{0:2}\t{1:20}\t{2:10}".format("No", "Mean", "Std"))
for column_name in features_name[0:3]:  # for all the columns except the last one
  column_mean= banknote_data_set[column_name].mean()
  print("{0:2d}\t{1}".format( features_name.index(column_name) + 1, column_mean), end="\t")

  column_std= banknote_data_set[column_name].std()
  print("{}".format(column_std))

  banknote_data_set[column_name]= (banknote_data_set[column_name]-column_mean)/column_std

banknote_data_set.head(5)

No	Mean                	Std       
 1	0.4337352570699707	2.842762586278562
 2	1.9223531206393603	5.869046743695522
 3	1.3976271172667651	4.310030090106595


,Variance,Skewness,Curtosis,Entropy,Class
0,1.121397,1.149036,-0.975614,-0.44699,0
1,1.446538,1.064065,-0.894710,-1.46210,0
2,1.207369,-0.777069,0.122174,0.10645,0
3,1.063355,1.295005,-1.254940,-3.59440,0
4,-0.036758,-1.086642,0.736462,-0.98880,0


## Spliting the features and class colum
    The imported data have all it's values in the same data frame table witch is hard to work with, so we are spliting it into two parts one is for features an another is for output classes.

## Converting The Data Frame to ndArray
    Working with pandas data frame is problematic and it may produce inconcistency with sklearn packages so we are converting the Data Frame to ndArray.

In [9]:
features= banknote_data_set[features_name[0:3]].to_numpy()
classes= banknote_data_set[features_name[-1]].to_numpy()

print("Features ndArray: ", features, end="\n\n")
print("Classes ndArray: ", classes)

Features ndArray:  [[ 1.12139676  1.14903615 -0.97561433]
 [ 1.44653822  1.06406494 -0.89471002]
 [ 1.20736947 -0.7770688   0.12217383]
 ...
 [-1.47182015 -2.62069017  3.75764729]
 [-1.40617978 -1.75583081  2.55111279]
 [-1.04674069 -0.43966137  0.29850671]]

Classes ndArray:  [0 0 0 ... 1 1 1]


## Train-Test Set Split
    To evaluate how the classifiers perform on the priviously unseen data we are going to split our original dataset into two sets one for training purpose and another for testing purpose.

In [10]:
train_features, test_features, train_classes, test_classes= train_test_split(features, classes, test_size= 0.3)

print("Train Features: \n\n", train_features, end="\n\n")
print("Train Classes: \n\n", train_classes, end="\n\n")
print("Test Features: \n\n", test_features, end="\n\n")
print("Test Classes: \n\n", test_classes)

Train Features: 

 [[-1.05666061 -1.49386323  1.7638561 ]
 [ 1.53469191  0.86275457 -0.7875414 ]
 [ 1.22516905  1.41875627 -1.2738257 ]
 ...
 [ 1.33295153 -0.13468169 -0.1567523 ]
 [ 0.42686109 -0.24539302  0.77662402]
 [ 0.35675323 -1.15891957  1.61139777]]

Train Classes: 

 [1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 0 1 0 0 0 0 0 0 1 1 0 0 0 1 0
 0 1 0 0 0 1 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 1
 1 1 0 1 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 1 0 0 0 0 1 0 1 1 0 0 0 1 1 0 0 0
 0 1 1 0 1 1 0 0 1 1 1 0 0 0 1 1 0 1 1 0 1 0 1 0 0 1 0 0 1 0 1 0 1 1 0 0 1
 1 0 1 1 0 1 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 1 0 1 0 1 1 1 0 1 1 1 0 0 0 0 0
 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0
 0 0 0 0 0 1 1 0 0 0 1 1 1 1 1 0 0 0 0 0 1 0 1 1 0 0 0 0 1 1 1 0 0 0 0 0 0
 0 1 1 1 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 0 1 0 0 1 0 1 0
 1 0 1 1 0 1 0 1 0 0 1 1 1 1 1 1 1 0 0 1 0 0 1 1 1 0 1 0 1 0 1 1 1 0 0 0 1
 0 1 0 1 1 0 0 1 1 0 1 0 1 0 0 0 1 1 0 0 1 0 0 

# K-Nearest Neighbour
    KNN is a non-parametric and lazy learning algorithm. Non-parametric means there is no assumption for underlying data distribution. In other words, the model structure determined from the dataset. This will be very helpful in practice where most of the real world datasets do not follow mathematical theoretical assumptions. Lazy algorithm means it does not need any training data points for model generation. All training data used in the testing phase. This makes training faster and testing phase slower and costlier. Costly testing phase means time and memory. In the worst case, KNN needs more time to scan all data points and scanning all data points will require more memory for storing training data.

## Working Principle
    In KNN, K is the number of nearest neighbors. The number of neighbors is the core deciding factor. K is generally an odd number if the number of classes is 2. When K=1, then the algorithm is known as the nearest neighbor algorithm. This is the simplest case.
    
![KNN](img/Knn_k1_z96jba.png)
    
    Suppose P1 is the point, for which label needs to predict. First, you find the k closest point to P1 and then classify points by majority vote of its k neighbors. Each object votes for their class and the class with the most votes is taken as the prediction. For finding closest similar points, you find the distance between points using distance measures such as Euclidean distance, Hamming distance, Manhattan distance and Minkowski distance. KNN has the following basic steps:
    
    1. Calculate distance
    2. Find closest neighbors
    3. Vote for labels

## Creating The Model

In [11]:
test_set_no,_ = train_features.shape # geting Training set size
print("Test set size:", test_set_no)

x_neighbors= list(range(1, (test_set_no//2))) # List of all the possible K values
print("No of K Neighbors:", len(x_neighbors))

Test set size: 960
No of K Neighbors: 479


### Evaluating The Best Value of K

In [50]:
knn_params= {
    'n_neighbors':x_neighbors
}

knn_model= KNeighborsClassifier(n_jobs=-1)
knn_grid= GridSearchCV(knn_model, knn_params, n_jobs= -1, verbose=True)

knn_grid.fit(train_features, train_classes)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 3 folds for each of 479 candidates, totalling 1437 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 828 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 1414 out of 1437 | elapsed:   13.8s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 1437 out of 1437 | elapsed:   14.1s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=-1,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=-1,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30, ...]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=True)

In [51]:
print(knn_grid.best_params_)

{'n_neighbors': 2}


### Final Model
    From the above plot we can observe that at ** k=2 ** the accuracy of Training set and Testing set is nearly the same. so finally we are training our model with ** k=2 **

In [47]:
knn_model= KNeighborsClassifier(n_neighbors= 2) # creating the classifier object
knn_model.fit(train_features, train_classes) # Training the model

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

## Evaluating Score on the TestSet

In [48]:
test_predict_knn= knn_model.predict(test_features) # Testing the model on Test set
print(classification_report(test_classes, test_predict_knn))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       234
           1       1.00      1.00      1.00       178

    accuracy                           1.00       412
   macro avg       1.00      1.00      1.00       412
weighted avg       1.00      1.00      1.00       412



# Support Vector Machine
    SVM offers very high accuracy compared to other classifiers such as logistic regression, and decision trees. It is known for its kernel trick to handle nonlinear input spaces.
    
    SVM constructs a hyperplane in multidimensional space to separate different classes. SVM generates optimal hyperplane in an iterative manner, which is used to minimize an error. The core idea of SVM is to find a maximum marginal hyperplane(MMH) that best divides the dataset into classes.
![SVM](img/index3_souoaz.png)

## Basic Terms
    There are some basic terms involved with svm. Here we discussedd all of them.

### Support Vectors
    Support vectors are the data points, which are closest to the hyperplane. These points will define the separating line better by calculating margins. These points are more relevant to the construction of the classifier.

### Hyperplane
    A hyperplane is a decision plane which separates between a set of objects having different class memberships.

### Margin
    A margin is a gap between the two lines on the closest class points. This is calculated as the perpendicular distance from the line to support vectors or closest points. If the margin is larger in between the classes, then it is considered a good margin, a smaller margin is a bad margin.

## Nonlinear data
![nonLinear_SVM](img/plot_circle_01.png)

    It’s pretty clear that there’s not a linear decision boundary (a single straight line that separates both tags). However, the vectors are very clearly segregated and it looks as though it should be easy to separate them.
    
    we will add a third dimension. Up until now we had two dimensions: x and y. We create a new z dimension, and we rule that it be calculated a certain way that is convenient for us: z = x² + y²
![nonLinear_SVM_3rdDimention](img/plot_circle_02.png)

    Note that since we are in three dimensions now, the hyperplane is a plane parallel to the x axis at a certain z (let’s say z = 1).

## The kernel trick
    The SVM algorithm is implemented in practice using a kernel. A kernel transforms an input data space into the required form. SVM uses a technique called the kernel trick. Here, the kernel takes a low-dimensional input space and transforms it into a higher dimensional space.

### Linear Kernel
    A linear kernel can be used as normal dot product any two given observations. The product between two vectors is the sum of the multiplication of each pair of input values.
    
    K(x, xi) = sum(x * xi)

### Polynomial Kernel
    A polynomial kernel is a more generalized form of the linear kernel. The polynomial kernel can distinguish curved or nonlinear input space.
    
    K(x,xi) = 1 + sum(x * xi)^d
    
    Where d is the degree of the polynomial. d=1 is similar to the linear transformation. The degree needs to be manually specified in the learning algorithm.

### Radial Basis Function Kernel
     RBF can map an input space in infinite dimensional space.
     
     K(x,xi) = exp(-gamma * sum((x – xi^2))
     
     Here gamma is a parameter, which ranges from 0 to 1. A higher value of gamma will perfectly fit the training dataset, which causes over-fitting. Gamma=0.1 is considered to be a good default value. The value of gamma needs to be manually specified in the learning algorithm.

## Creating The Model

In [68]:
x_c_values= list(np.arange(0, 20, 0.03)[1:])

print("Testing C values(first 5): ", x_c_values[0:5])

Testing C values(first 5):  [0.03, 0.06, 0.09, 0.12, 0.15]


### Evaluating Hyperparameter

In [69]:
svm_params= {
    'C':x_c_values,
    'kernel':[ 'linear', 'poly', 'rbf', 'sigmoid'],
    'gamma':['scale', 'auto'],
}

svm_model= svm.SVC(verbose= True)
svm_grid= GridSearchCV(svm_model, svm_params, n_jobs= -1, verbose=True)

svm_grid.fit(train_features, train_classes)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 3 folds for each of 5328 candidates, totalling 15984 fits


[Parallel(n_jobs=-1)]: Done  84 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 4584 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 12084 tasks      | elapsed:    5.6s


[LibSVM]

[Parallel(n_jobs=-1)]: Done 15984 out of 15984 | elapsed:    7.1s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=True),
             iid='warn', n_jobs=-1,
             param_grid={'C': [0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21, 0.24,
                               0.27, 0.3, 0.32999999999999996, 0.36, 0.39, 0.42,
                               0.44999999999999996, 0.48, 0.51, 0.54, 0.57, 0.6,
                               0.63, 0.6599999999999999, 0.69, 0.72, 0.75, 0.78,
                               0.8099999999999999, 0.84, 0.87,
                               0.8999999999999999, ...],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly

In [70]:
print(svm_grid.best_params_)

{'C': 11.16, 'gamma': 'scale', 'kernel': 'rbf'}


### Final Model
    From the above plot we can observe that at c=2.04 the accuracy of Training set and Testing set is nearly the same. so finally we are training our model with ** k=2.04 **

In [71]:
svm_model= svm.SVC(C=11.16, kernel='rbf', gamma= 'scale') # Creating SVM object
svm_model.fit(train_features, train_classes) # Training the model on training set

SVC(C=11.16, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

## Evaluating Accuracy on TestSet

In [72]:
test_predict_svm= svm_model.predict(test_features) # Predicting on Test set
print(classification_report(test_classes, test_predict_svm))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       234
           1       1.00      1.00      1.00       178

    accuracy                           1.00       412
   macro avg       1.00      1.00      1.00       412
weighted avg       1.00      1.00      1.00       412



# Neural Network
    Neural networks are created by adding the layers of these perceptrons together, known as a multi-layer perceptron model. There are three layers of a neural network - the input, hidden, and output layers. The input layer directly receives the data, whereas the output layer creates the required output. The layers in between are known as hidden layers where the intermediate computation takes place.
    

## Feed-Forward
    Following are the steps performed during the feed-forward phase:
    
    1. The values received in the input layer are multiplied with the weights. A bias is added to the summation of the inputs and weights in order to avoid null values.
    
    2. Each neuron in the first hidden layer receives different values from the input layer depending upon the weights and bias. Neurons have an activation function that operates upon the value received from the input layer. The activation function can be of many types, like a step function, sigmoid function, relu function, or tanh function. As a rule of thumb, relu function is used in the hidden layer neurons and sigmoid function is used for the output layer neuron.
    
    3. The outputs from the first hidden layer neurons are multiplied with the weights of the second hidden layer; the results are summed together and passed to the neurons of the proceeding layers. This process continues until the outer layer is reached. The values calculated at the outer layer are the actual outputs of the algorithm.

## Network Architecture

![Neural_Network](img/ann-in-hidden-out.jpg)

## Back Propagation
    Back propagation phase consists of the following steps:
    
![Back_Propagation_Step](img/tikz21.png)

    The backpropagation equations provide us with a way of computing the gradient of the cost function. Let's explicitly write this out in the form of an algorithm:

**Input x:** Set the corresponding activation a1 for the input layer.

**Feedforward:** For each l=2,3,…,L compute 

                    zl=wlal−1+bl and al=σ(zl)
                    
**Output error δL:** Compute the vector

                    δL=∇aC⊙σ′(zL)
                    
**Backpropagate the error:** For each l=L−1,L−2,…,2 compute 

                    δl=((wl+1)Tδl+1)⊙σ′(zl)
                    
**Output:** The gradient of the cost function is given by 

                    ∂C∂wljk=al−1kδlj and ∂C∂blj=δlj

## Creating The Model

In [75]:
nn_params= {
    'activation':['identity', 'logistic', 'tanh', 'relu'],
    'solver':[ 'lbfgs', 'sgd', 'adam'],
    'learning_rate':['constant', 'invscaling', 'adaptive'],
    'hidden_layer_sizes':[(100, 20), (20, 20), (15,)]
}

nn_model= MLPClassifier(verbose= True)
nn_grid= GridSearchCV(nn_model, nn_params, n_jobs= -1, verbose=True)

nn_grid.fit(train_features, train_classes)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 3 folds for each of 108 candidates, totalling 324 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 324 out of 324 | elapsed:   10.1s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_iter=200,
                                     momentum=0.9, n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     rando...
                                     validation_fraction=0.1, verbose=True,
                                     warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'activation': ['identity', 'logistic', 'tanh', 'relu'],
                         'hidden_layer_sizes': [(100, 20), (20, 20), (1

In [76]:
print(nn_grid.best_params_)

{'activation': 'logistic', 'hidden_layer_sizes': (100, 20), 'learning_rate': 'adaptive', 'solver': 'lbfgs'}


In [79]:
nn_model= MLPClassifier(verbose= True, hidden_layer_sizes= (100,20), activation= 'logistic', solver= 'lbfgs', learning_rate='adaptive') # Creating neural network object
nn_model.fit(train_features, train_classes)

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
              beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100, 20), learning_rate='adaptive',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=True, warm_start=False)

### Evaluating Accuracy on TestSet

In [80]:
test_predict_nn= nn_model.predict(test_features)
print(classification_report(test_classes, test_predict_nn))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       234
           1       1.00      1.00      1.00       178

    accuracy                           1.00       412
   macro avg       1.00      1.00      1.00       412
weighted avg       1.00      1.00      1.00       412



# Conclusion

    After analyzing various techniques used to detect forged banknotes, this paper presents banknote authentication for recognizing the banknote as genuine or fake by using three supervised learning techniques. Extensive experiments have been performed on banknotes dataset using both the models to find the best model suitable for classification of the notes.
    
    From this discussion we can see that all the models are perfect interms of accuracy but the neural network model perform best incase of testing speed but training speed of neural network is slow where in KNN we don't have any training phase. SVM is inoverall the best sutable classifier for this type of data because it have a proper balance between training speed and testing speed.